### Notebook to configure model

In [1]:
import time

from models.modules.multihead import *
from utils.prepare_data import *

import pandas as pd


/anaconda3/envs/mlenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [72]:

# Hyperparameter
MAX_SEQ_LENGTH = 100
EMBEDDING_SIZE = 128
HIDDEN_SIZE = 512
ATTENTION_SIZE = 64
lr = 1e-3
BATCH_SIZE = 256
KEEP_PROB = 0.5
LAMBDA = 0.0001

vocab_size = 5

MAX_LABEL = 2


In [92]:
# x_train_0 = pd.read_csv('../data/NACTG-train-normal-BRAF.txt', sep=',',header=None)
# x_train_1 = pd.read_csv('../data/NACTG-train-tumor-BRAF.txt', sep=',',header=None)
# x_test_0 = pd.read_csv('../data/NACTG-test-normal-BRAF.txt', sep=',',header=None)
# x_test_1 = pd.read_csv('../data/NACTG-test-tumor-BRAF.txt', sep=',',header=None)

# x_train_0.head()

# load data
x_train, y_train = load_data("../data/train-BRAF.csv", sample_ratio=1)
x_test, y_test = load_data("../data/test-BRAF.csv", sample_ratio=1)


(2440198, 2)
(155244, 2)


In [58]:
base_number = {'N':0, 'A':1, 'C':2, 'T':3, 'G':4}
x_train_l = np.ndarray((len(x_train),MAX_SEQ_LENGTH))

for t in np.arange(len(x_train)):
    line = list(x_train[t])[1:MAX_SEQ_LENGTH+1]
    for k in np.arange(MAX_SEQ_LENGTH):
        x_train_l[t,k] = base_number[line[k]]
    
x_test_l = np.ndarray((len(x_test),MAX_SEQ_LENGTH))

for t in np.arange(len(x_test)):    
    line = list(x_test[t])[1:MAX_SEQ_LENGTH+1]
    for k in np.arange(MAX_SEQ_LENGTH):
        x_test_l[t,k] = base_number[line[k]]
    
print(x_train_l.shape)
print(x_test_l.shape)
  

(2440198, 100)
(155244, 100)


In [86]:
print(x_train_l.shape)
print(x_test_l.shape)

print(x_train_l[0])

(2440198, 100)
(155244, 100)
[3. 3. 1. 4. 3. 3. 1. 1. 1. 1. 1. 3. 4. 1. 4. 4. 3. 3. 4. 4. 1. 1. 1. 4.
 3. 1. 1. 1. 3. 2. 3. 4. 1. 2. 2. 3. 1. 4. 1. 3. 4. 1. 3. 3. 4. 4. 3. 3.
 3. 4. 1. 2. 1. 1. 3. 4. 1. 4. 4. 1. 3. 3. 1. 1. 2. 3. 3. 3. 1. 2. 3. 1.
 1. 3. 3. 1. 4. 1. 3. 3. 1. 3. 4. 3. 4. 4. 2. 3. 4. 1. 3. 4. 4. 4. 3. 4.
 3. 3. 3. 3.]


In [ ]:
x_train = x_train_l[0:10000,:]
x_test = x_test_l[0:1000,:]

y_train = y_train[0:10000,:]
y_test = y_test[0:1000,:]

print(x_train.shape)
print(x_test.shape)

In [88]:
# # data preprocessing
# x_train, x_test, vocab, vocab_size = \
#     data_preprocessing(x_train, x_test, MAX_SEQ_LENGTH)
# print(vocab_size)


In [94]:
# split dataset to test and dev
x_test, x_dev, y_test, y_dev, dev_size, test_size = \
    split_dataset(x_test, y_test, 0.1)
print("Validation size: ", dev_size)


1000
100
Validation size:  100


In [95]:

graph = tf.Graph()
with graph.as_default():

    batch_x = tf.placeholder(tf.int32, [None, MAX_SEQ_LENGTH])
    batch_y = tf.placeholder(tf.float32, [None, MAX_LABEL])
    keep_prob = tf.placeholder(tf.float32)

    embeddings_var = tf.Variable(tf.random_uniform([vocab_size, EMBEDDING_SIZE], -1.0, 1.0), trainable=True)
    batch_embedded = tf.nn.embedding_lookup(embeddings_var, batch_x)
    # multihead attention
    outputs = multihead_attention(queries=batch_embedded, keys=batch_embedded)
    # FFN(x) = LN(x + point-wisely NN(x))
    outputs = feedforward(outputs, [HIDDEN_SIZE, EMBEDDING_SIZE])
    print(outputs.shape)
    outputs = tf.reshape(outputs, [-1, MAX_SEQ_LENGTH * EMBEDDING_SIZE])
    logits = tf.layers.dense(outputs, units=MAX_LABEL)
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=batch_y))
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

    # Accuracy metric
    prediction = tf.argmax(tf.nn.softmax(logits), 1)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, tf.argmax(batch_y, 1)), tf.float32))



Conv ret: (?, 100, 128)
(?, 100, 128)


In [96]:

epochs = 5

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    print("Initialized! ")

    print("Start trainning")
    start = time.time()
    for e in range(epochs):

        epoch_start = time.time()
        print("Epoch %d start !" % (e + 1))
        for x_batch, y_batch in fill_feed_dict(x_train, y_train, BATCH_SIZE):
            fd = {batch_x: x_batch, batch_y: y_batch, keep_prob: KEEP_PROB}
            l, _, acc = sess.run([loss, optimizer, accuracy], feed_dict=fd)

        epoch_finish = time.time()
        print("Validation accuracy and loss: ", sess.run([accuracy, loss], feed_dict={
            batch_x: x_dev,
            batch_y: y_dev,
            keep_prob: 1.0
        }))
        print("epoch time:", epoch_finish - epoch_start , " s")

    print("Training finished, time consumed : ", time.time() - start, " s")
    print("start predicting:  \n")
    test_accuracy = sess.run([accuracy], feed_dict={batch_x: x_test, batch_y: y_test, keep_prob: 1})
    print("Test accuracy : %f %%" % (test_accuracy[0] * 100))



Initialized! 
Start trainning
Epoch 1 start !
Validation accuracy and loss:  [0.47, 0.80978376]
epoch time: 172.9900119304657  s
Epoch 2 start !
Validation accuracy and loss:  [0.46, 0.72138095]
epoch time: 187.37515902519226  s
Epoch 3 start !
Validation accuracy and loss:  [0.5, 0.76346606]
epoch time: 152.2712278366089  s
Epoch 4 start !
Validation accuracy and loss:  [0.6, 0.69934523]
epoch time: 439.39326095581055  s
Epoch 5 start !
Validation accuracy and loss:  [0.38, 0.80945414]
epoch time: 121.4345600605011  s
Training finished, time consumed :  1077.1485340595245  s
start predicting:  

Test accuracy : 42.444444 %
